## Example code for a convolutional layer (without biases), and its backward pass

In [1]:
# extend conv2 to accept "full" or "valid" as an third argument
function Base.conv2(A, B, shape::String)   
    if shape == "full"
        return Base.conv2(A,B)
    elseif shape == "valid"
        ranges = [ min(a,b):max(a,b) for (a,b) in zip(size(A),size(B)) ]
        return Base.conv2(A,B)[ranges...]
    else
        error("shape must be full or valid")
    end
end

In [2]:
f(x) = tanh(x)        # substitute your favorite activation function here
df(y) = 1 - y.*y

df (generic function with 1 method)

In [3]:
# forward pass
nin = 3     # number of input images/feature maps
nout = 4    # number of output images/feature maps
in = rand(16,16,nin)
w = rand(3,3,nout,nin)
outsize = (size(in,1)-size(w,1)+1, size(in,2)-size(w,2)+1, size(w,3))

out = zeros(outsize)       # will accumulate to this array
for i = 1:nout
    for j = 1:nin
        out[:,:,i] += conv2(w[:,:,i,j], in[:,:,j],"valid")
    end
end
out = f(out);

In [4]:
# backward pass
deltaout = rand(size(out))   # input to the backward pass, so name could be confusing

deltain = zeros(size(in))     # will accumulate to this array
for j = 1:nin
    for i = 1:nout
        deltain[:,:,j] += conv2(deltaout[:,:,i], w[end:-1:1,end:-1:1,i,j],"full")
    end
end
deltain .*= df(in);

In [5]:
# kernel update
for i = 1:nout
    for j = 1:nin
        w[:,:,i,j] += conv2(deltaout[:,:,i], in[end:-1:1,end:-1:1,j],"valid")
    end
end

### For Whoever Shares, to Him More Gradient Will Be Given.  (corruption of Mark 4:25)